
Complete E-Commerce Recommendation System - Data Processing & API
This script processes retail data and provides a REST API for product recommendations


# ===================================================
# PART 1: SETUP AND IMPORTS
# ===================================================

In [4]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [5]:
!pip install -q flask flask-cors pyngrok mlxtend openpyxl scikit-learn

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
from datetime import datetime, timezone
import time
import threading

# ML Libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA

# Association Rules
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Web Framework
from flask import Flask, request, jsonify, make_response
from flask_cors import CORS
from functools import lru_cache
import json

print("✅ All packages imported successfully")

✅ All packages imported successfully


# ===================================================
# PART 2: DATA LOADING AND PREPROCESSING
# ===================================================

In [6]:
def download_and_load_data():
    """Download Online Retail dataset from UCI repository"""
    print("=" * 80)
    print("DOWNLOADING ONLINE RETAIL DATASET")
    print("=" * 80)

    # Download dataset
    !wget -q https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx -O online_retail.xlsx

    # Load Excel file
    print("Loading Online Retail dataset...")
    df = pd.read_excel('online_retail.xlsx')
    print(f"✅ Loaded {len(df)} transactions")

    # Save as CSV for faster loading
    df.to_csv('online_retail_raw.csv', index=False)
    print("✅ Saved raw data as CSV")

    return df

In [7]:
def preprocess_retail_data(df):
    """Complete preprocessing pipeline for retail data"""
    print("\n" + "=" * 80)
    print("DATA PREPROCESSING PIPELINE")
    print("=" * 80)

    # 1. Remove missing CustomerID
    print("\n1. Handling missing values...")
    initial_shape = df.shape
    df = df.dropna(subset=['CustomerID'])
    print(f"   Removed {initial_shape[0] - df.shape[0]} rows with missing CustomerID")

    # 2. Filter positive quantities (remove returns)
    df = df[df['Quantity'] > 0]
    df = df[df['UnitPrice'] > 0]
    print(f"   Removed negative quantities. Shape: {df.shape}")

    # 3. Remove duplicates
    df = df.drop_duplicates()
    print(f"   Removed duplicates. Shape: {df.shape}")

    # 4. Create TotalPrice feature
    df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

    # 5. Convert date and standardize columns
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    df['CustomerID'] = df['CustomerID'].astype(str).str.split('.').str[0]

    # 6. Add time-based features
    df['Year'] = df['InvoiceDate'].dt.year
    df['Month'] = df['InvoiceDate'].dt.month
    df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
    df['Hour'] = df['InvoiceDate'].dt.hour

    # 7. Remove outliers using IQR
    print("\n2. Removing outliers...")
    Q1 = df['TotalPrice'].quantile(0.25)
    Q3 = df['TotalPrice'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df['TotalPrice'] >= lower_bound) & (df['TotalPrice'] <= upper_bound)]

    print(f"✅ Final preprocessed shape: {df.shape}")

    # Save preprocessed data
    df.to_csv('online_retail_preprocessed.csv', index=False)
    print("✅ Saved preprocessed data")

    return df

In [8]:
def create_customer_features(df):
    """Create RFM features for customers"""
    print("\n" + "=" * 80)
    print("CREATING CUSTOMER FEATURES (RFM)")
    print("=" * 80)

    max_date = df['InvoiceDate'].max()

    customer_features = df.groupby('CustomerID').agg({
        'InvoiceDate': lambda x: (max_date - x.max()).days,
        'InvoiceNo': 'nunique',
        'TotalPrice': 'sum',
        'Quantity': 'sum',
        'StockCode': 'nunique'
    }).reset_index()

    customer_features.columns = ['CustomerID', 'Recency', 'Frequency',
                                 'Monetary', 'TotalItems', 'UniqueProducts']

    customer_features['AvgOrderValue'] = customer_features['Monetary'] / customer_features['Frequency']

    print(f"✅ Created features for {len(customer_features)} customers")
    customer_features.to_csv('customer_features.csv', index=False)

    return customer_features

In [9]:
def create_product_features(df):
    """Create product-level features"""
    print("\n" + "=" * 80)
    print("CREATING PRODUCT FEATURES")
    print("=" * 80)

    product_features = df.groupby(['StockCode', 'Description']).agg({
        'Quantity': 'sum',
        'TotalPrice': 'sum',
        'CustomerID': 'nunique',
        'InvoiceNo': 'nunique',
        'UnitPrice': 'mean'
    }).reset_index()

    product_features.columns = ['StockCode', 'Description', 'TotalQuantity',
                                'TotalRevenue', 'UniqueCustomers', 'TotalOrders', 'AvgPrice']

    product_features['PopularityScore'] = (
        product_features['TotalQuantity'] * 0.3 +
        product_features['UniqueCustomers'] * 0.4 +
        product_features['TotalOrders'] * 0.3
    )

    print(f"✅ Created features for {len(product_features)} products")
    product_features.to_csv('product_features.csv', index=False)

    return product_features


# ===================================================
# PART 3: CLUSTERING (K-MEANS)
# ===================================================

In [10]:
def perform_customer_clustering(customer_features):
    """Perform customer segmentation using clustering"""
    print("\n" + "=" * 80)
    print("CUSTOMER CLUSTERING")
    print("=" * 80)

    feature_cols = ['Recency', 'Frequency', 'Monetary', 'TotalItems', 'UniqueProducts']
    X = customer_features[feature_cols].values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA(n_components=2, random_state=42)
    X_reduced = pca.fit_transform(X_scaled)

    print("\n1. K-Means Clustering...")
    kmeans = KMeans(n_clusters=4, random_state=42, n_init='auto')
    kmeans_labels = kmeans.fit_predict(X_scaled)
    customer_features['KMeans_Cluster'] = kmeans_labels

    cluster_summary = customer_features.groupby('KMeans_Cluster').agg({
        'Recency': 'mean',
        'Frequency': 'mean',
        'Monetary': 'mean'
    })

    cluster_names = []
    for idx, row in cluster_summary.iterrows():
        if row['Monetary'] > cluster_summary['Monetary'].median():
            if row['Frequency'] > cluster_summary['Frequency'].median():
                cluster_names.append('VIP Customers')
            else:
                cluster_names.append('High Value Customers')
        else:
            if row['Frequency'] > cluster_summary['Frequency'].median():
                cluster_names.append('Loyal Customers')
            else:
                cluster_names.append('New Customers')

    cluster_label_map = dict(zip(cluster_summary.index, cluster_names))
    customer_features['Cluster_Label'] = customer_features['KMeans_Cluster'].map(cluster_label_map)

    print(f"✅ K-Means: Found {len(np.unique(kmeans_labels))} clusters")

    customer_features.to_csv('customer_clusters.csv', index=False)
    print("✅ Saved clustering results")

    os.makedirs('models', exist_ok=True)
    with open('models/kmeans_model.pkl', 'wb') as f:
        pickle.dump(kmeans, f)
    with open('models/scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)
    with open('models/pca.pkl', 'wb') as f:
        pickle.dump(pca, f)

    return customer_features, scaler, pca

# ===================================================
# PART 4: ASSOCIATION RULES (FP-GROWTH)
# ===================================================

In [11]:
def generate_association_rules(df):
    """Generate association rules for market basket analysis"""
    print("\n" + "=" * 80)
    print("ASSOCIATION RULE MINING")
    print("=" * 80)

    print("Creating transaction matrix...")
    basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().fillna(0)

    def encode_units(x):
        return 1 if x > 0 else 0

    basket_sets = basket.applymap(encode_units)

    print("Generating frequent itemsets with FP-Growth...")
    frequent_itemsets = fpgrowth(basket_sets, min_support=0.01, use_colnames=True)

    print("Generating association rules...")
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
    rules = rules[rules['confidence'] >= 0.2]
    rules = rules.sort_values('lift', ascending=False)

    print(f"✅ Generated {len(rules)} association rules")

    rules.to_csv('association_rules.csv', index=False)
    print("✅ Saved association rules")

    return rules

# ===================================================
# PART 5: CLASSIFICATION MODELS
# ===================================================

In [12]:
def train_classification_models(df, customer_features):
    """Train models to classify transactions as high/low value"""
    print("\n" + "=" * 80)
    print("TRAINING CLASSIFICATION MODELS")
    print("=" * 80)

    transaction_features = df.groupby('InvoiceNo').agg({
        'TotalPrice': 'sum',
        'Quantity': 'sum',
        'StockCode': 'nunique',
        'CustomerID': 'first'
    }).reset_index()

    transaction_features = transaction_features.merge(
        customer_features[['CustomerID', 'KMeans_Cluster']],
        on='CustomerID',
        how='left'
    )

    median_value = transaction_features['TotalPrice'].median()
    transaction_features['Label'] = (transaction_features['TotalPrice'] > median_value).astype(int)

    feature_cols = ['Quantity', 'StockCode', 'KMeans_Cluster']
    X = transaction_features[feature_cols].fillna(0).values
    y = transaction_features['Label'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("\n1. Training Decision Tree...")
    dt_classifier = DecisionTreeClassifier(random_state=42, max_depth=10)
    dt_classifier.fit(X_train, y_train)

    y_pred_dt = dt_classifier.predict(X_test)
    print(f"   Accuracy: {accuracy_score(y_test, y_pred_dt):.4f}")

    print("\n2. Training K-Nearest Neighbors...")
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    knn_classifier.fit(X_train, y_train)

    y_pred_knn = knn_classifier.predict(X_test)
    print(f"   Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")

    with open('models/dt_classifier.pkl', 'wb') as f:
        pickle.dump(dt_classifier, f)
    with open('models/knn_classifier.pkl', 'wb') as f:
        pickle.dump(knn_classifier, f)

    print("✅ Models saved successfully")

    return dt_classifier, knn_classifier

# =================================================
# PART 6: OLAP OPERATIONS
# =================================================

In [13]:
def perform_olap_operations(df):
    """Perform OLAP operations on the data"""
    print("\n" + "=" * 80)
    print("OLAP OPERATIONS")
    print("=" * 80)

    # Slice by country
    def slice_by_country(country):
        return df[df['Country'] == country]

    # Dice by multiple dimensions
    def dice_operation(countries, date_range=None):
        result = df[df['Country'].isin(countries)]
        if date_range:
            result = result[(result['InvoiceDate'] >= date_range[0]) &
                          (result['InvoiceDate'] <= date_range[1])]
        return result

    # Roll-up aggregations
    country_rollup = df.groupby('Country').agg({
        'TotalPrice': 'sum',
        'InvoiceNo': 'nunique',
        'CustomerID': 'nunique',
        'Quantity': 'sum'
    }).round(2).sort_values('TotalPrice', ascending=False)

    customer_rollup = df.groupby('CustomerID').agg({
        'TotalPrice': 'sum',
        'InvoiceNo': 'nunique',
        'Quantity': 'sum',
        'StockCode': 'nunique'
    }).round(2).sort_values('TotalPrice', ascending=False)

    product_rollup = df.groupby(['StockCode', 'Description']).agg({
        'TotalPrice': 'sum',
        'Quantity': 'sum',
        'CustomerID': 'nunique',
        'UnitPrice': 'mean'
    }).round(2).sort_values('TotalPrice', ascending=False)

    # Save OLAP results
    country_rollup.to_csv('olap_country.csv')
    customer_rollup.to_csv('olap_customer.csv')
    product_rollup.to_csv('olap_product.csv')

    print("✅ OLAP operations completed and saved")

    return country_rollup, customer_rollup, product_rollup

# ==================================================
# PART 7: MAIN PROCESSING PIPELINE
# ==================================================

In [14]:
def run_complete_pipeline():
    """Execute the complete data processing pipeline"""
    print("\n" + "=" * 80)
    print("E-COMMERCE RECOMMENDATION SYSTEM - COMPLETE PIPELINE")
    print("=" * 80)

    # 1. Load Data
    df = download_and_load_data()

    # 2. Preprocess Data
    df = preprocess_retail_data(df)

    # 3. Create Features
    customer_features = create_customer_features(df)
    product_features = create_product_features(df)

    # 4. Perform Clustering
    customer_features, scaler, pca = perform_customer_clustering(customer_features)

    # 5. Generate Association Rules
    rules = generate_association_rules(df)

    # 6. Train Classification Models
    dt_model, knn_model = train_classification_models(df, customer_features)

    print("\n" + "=" * 80)
    print("✅ PIPELINE EXECUTION COMPLETE!")
    print("=" * 80)

    return {
        'df': df,
        'customer_features': customer_features,
        'product_features': product_features,
        'rules': rules,
        'models': {
            'dt': dt_model,
            'knn': knn_model,
            'scaler': scaler,
            'pca': pca
        }
    }

# Run the pipeline
pipeline_results = run_complete_pipeline()


E-COMMERCE RECOMMENDATION SYSTEM - COMPLETE PIPELINE
DOWNLOADING ONLINE RETAIL DATASET
Loading Online Retail dataset...
✅ Loaded 541909 transactions
✅ Saved raw data as CSV

DATA PREPROCESSING PIPELINE

1. Handling missing values...
   Removed 135080 rows with missing CustomerID
   Removed negative quantities. Shape: (397884, 8)
   Removed duplicates. Shape: (392692, 8)

2. Removing outliers...
✅ Final preprocessed shape: (361461, 13)
✅ Saved preprocessed data

CREATING CUSTOMER FEATURES (RFM)
✅ Created features for 4194 customers

CREATING PRODUCT FEATURES
✅ Created features for 3865 products

CUSTOMER CLUSTERING

1. K-Means Clustering...
✅ K-Means: Found 4 clusters
✅ Saved clustering results

ASSOCIATION RULE MINING
Creating transaction matrix...
Generating frequent itemsets with FP-Growth...
Generating association rules...
✅ Generated 804 association rules
✅ Saved association rules

TRAINING CLASSIFICATION MODELS

1. Training Decision Tree...
   Accuracy: 0.8852

2. Training K-Near

# ===================================================
# PART 8: FLASK API WITH NGROK
# ===================================================

In [ ]:
app = Flask(__name__)

# Configure CORS properly for ngrok
CORS(app, resources={
    r"/*": {
        "origins": "*",
        "allow_headers": ["Content-Type", "Authorization", "ngrok-skip-browser-warning"],
        "methods": ["GET", "POST", "PUT", "DELETE", "OPTIONS"]
    }
})

# Global variables
GLOBAL_DATA = {
    'df': None,
    'customer_features': None,
    'product_features': None,
    'association_rules': None,
    'dt_classifier': None,
    'knn_classifier': None
}

def load_all_data():
    """Load all processed data and models"""
    global GLOBAL_DATA

    print("\n" + "=" * 80)
    print("LOADING DATA FOR API")
    print("=" * 80)

    try:
        GLOBAL_DATA['df'] = pd.read_csv('online_retail_preprocessed.csv')
        GLOBAL_DATA['df']['InvoiceDate'] = pd.to_datetime(GLOBAL_DATA['df']['InvoiceDate'])
        GLOBAL_DATA['customer_features'] = pd.read_csv('customer_clusters.csv')
        GLOBAL_DATA['product_features'] = pd.read_csv('product_features.csv')
        GLOBAL_DATA['association_rules'] = pd.read_csv('association_rules.csv')

        with open('models/dt_classifier.pkl', 'rb') as f:
            GLOBAL_DATA['dt_classifier'] = pickle.load(f)
        with open('models/knn_classifier.pkl', 'rb') as f:
            GLOBAL_DATA['knn_classifier'] = pickle.load(f)

        print("✅ All data and models loaded successfully")
        return True

    except Exception as e:
        print(f"❌ Error loading data: {e}")
        return False

# API Routes with proper headers
@app.after_request
def after_request(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization,ngrok-skip-browser-warning')
    response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE,OPTIONS')
    return response

@app.route('/api/test', methods=['GET', 'OPTIONS'])
def test_connection():
    """Test API connection"""
    if request.method == 'OPTIONS':
        return make_response('', 200)

    return jsonify({
        'status': 'connected',
        'message': 'Connection successful!',
        'timestamp': datetime.now(timezone.utc).isoformat()
    })

@app.route('/api/stats', methods=['GET', 'OPTIONS'])
def get_stats():
    """Get overall statistics"""
    if request.method == 'OPTIONS':
        return make_response('', 200)

    try:
        df = GLOBAL_DATA['df']
        customer_features = GLOBAL_DATA['customer_features']
        product_features = GLOBAL_DATA['product_features']

        stats = {
            'total_transactions': int(len(df)),
            'total_customers': int(len(customer_features)),
            'total_products': int(len(product_features)),
            'total_revenue': float(df['TotalPrice'].sum())
        }

        return jsonify(stats)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/products', methods=['GET', 'OPTIONS'])
def get_products():
    """Get products with pagination and search"""
    if request.method == 'OPTIONS':
        return make_response('', 200)

    try:
        page = request.args.get('page', 1, type=int)
        per_page = request.args.get('per_page', 20, type=int)
        search = request.args.get('search', '')

        products = GLOBAL_DATA['product_features'].copy()

        if search:
            products = products[products['Description'].str.contains(search, case=False, na=False)]

        products = products.sort_values('PopularityScore', ascending=False)

        total = len(products)
        start = (page - 1) * per_page
        end = start + per_page

        products_page = products.iloc[start:end]

        return jsonify({
            'products': products_page.to_dict('records'),
            'total': total,
            'page': page,
            'per_page': per_page
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/recommend', methods=['POST', 'OPTIONS'])
def get_recommendations():
    """Get product recommendations"""
    if request.method == 'OPTIONS':
        return make_response('', 200)

    try:
        data = request.json
        stock_code = data.get('stock_code')
        n_recommendations = min(data.get('n_recommendations', 10), 20)

        df = GLOBAL_DATA['df']

        if stock_code:
            # Get transactions containing the product
            product_transactions = df[df['StockCode'] == stock_code]['InvoiceNo'].unique()

            # Get related products from those transactions
            related_products = df[df['InvoiceNo'].isin(product_transactions)]
            related_products = related_products[related_products['StockCode'] != stock_code]

            # Aggregate and rank
            top_related = related_products.groupby(['StockCode', 'Description']).agg({
                'Quantity': 'sum',
                'TotalPrice': 'sum',
                'InvoiceNo': 'nunique'
            }).nlargest(n_recommendations, 'Quantity')

            recommendations = []
            for idx, row in top_related.iterrows():
                recommendations.append({
                    'stock_code': idx[0],
                    'description': idx[1],
                    'confidence': min(0.9, row['InvoiceNo'] / len(product_transactions)),
                    'lift': 2.5
                })

            return jsonify({
                'recommendations': recommendations[:n_recommendations],
                'total': len(recommendations)
            })

        return jsonify({'recommendations': [], 'total': 0})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/customers/<customer_id>', methods=['GET', 'OPTIONS'])
def get_customer_profile(customer_id):
    """Get customer profile (includes recent purchases)"""
    if request.method == 'OPTIONS':
        return make_response('', 200)

    try:
        customer_features = GLOBAL_DATA['customer_features']
        df = GLOBAL_DATA['df']

        # Normalize customer id as string (preprocessing stores CustomerID as string)
        customer_id_str = str(customer_id)

        # Find customer in features (ensure string comparison)
        customer_row = customer_features[customer_features['CustomerID'].astype(str) == customer_id_str]

        if customer_row.empty:
            return jsonify({'error': 'Customer not found'}), 404

        profile = customer_row.iloc[0].to_dict()

        # Get recent transactions for this customer from the main dataframe
        customer_tx = df[df['CustomerID'].astype(str) == customer_id_str].copy()
        purchases = []
        if not customer_tx.empty:
            # Ensure InvoiceDate is datetime and sort by recent
            try:
                customer_tx['InvoiceDate'] = pd.to_datetime(customer_tx['InvoiceDate'])
                customer_tx = customer_tx.sort_values('InvoiceDate', ascending=False)
            except Exception:
                pass

            # Select relevant columns and limit rows
            cols = []
            for c in ['StockCode','Description','Quantity','Country','InvoiceDate','InvoiceNo']:
                if c in customer_tx.columns:
                    cols.append(c)
            if cols:
                purchases = customer_tx[cols].head(50).to_dict('records')
                # Convert any Timestamp to ISO strings
                for p in purchases:
                    if 'InvoiceDate' in p and p['InvoiceDate'] is not None:
                        try:
                            p['InvoiceDate'] = pd.to_datetime(p['InvoiceDate']).isoformat()
                        except Exception:
                            p['InvoiceDate'] = str(p['InvoiceDate'])

        return jsonify({'customer_id': customer_id_str, 'profile': profile, 'purchases': purchases})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/association_rules', methods=['GET', 'OPTIONS'])
def get_association_rules():
    """Get association rules"""
    if request.method == 'OPTIONS':
        return make_response('', 200)

    try:
        min_confidence = request.args.get('min_confidence', 0.5, type=float)
        min_lift = request.args.get('min_lift', 1.0, type=float)

        rules = GLOBAL_DATA['association_rules'].copy()

        # Filter rules
        rules = rules[
            (rules['confidence'] >= min_confidence) &
            (rules['lift'] >= min_lift)
        ]

        rules = rules.sort_values('lift', ascending=False)

        return jsonify({
            'rules': rules.head(50).to_dict('records'),
            'total': len(rules)
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/olap/slice', methods=['POST', 'OPTIONS'])
def olap_slice():
    """OLAP Slice operation"""
    if request.method == 'OPTIONS':
        return make_response('', 200)

    try:
        data = request.json
        country = data.get('country')

        df = GLOBAL_DATA['df']
        sliced_data = df[df['Country'] == country]

        result = {
            'total_transactions': int(len(sliced_data)),
            'total_revenue': float(sliced_data['TotalPrice'].sum()),
            'unique_customers': int(sliced_data['CustomerID'].nunique()),
            'total_quantity': int(sliced_data['Quantity'].sum())
        }

        return jsonify(result)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/olap/dice', methods=['POST', 'OPTIONS'])
def olap_dice():
    """OLAP Dice operation"""
    if request.method == 'OPTIONS':
        return make_response('', 200)

    try:
        data = request.json
        countries = data.get('countries', [])
        date_range = data.get('date_range')

        df = GLOBAL_DATA['df']
        diced_data = df[df['Country'].isin(countries)]

        if date_range:
            start_date = pd.to_datetime(date_range[0])
            end_date = pd.to_datetime(date_range[1])
            diced_data = diced_data[
                (diced_data['InvoiceDate'] >= start_date) &
                (diced_data['InvoiceDate'] <= end_date)
            ]

        result = {
            'total_transactions': int(len(diced_data)),
            'total_revenue': float(diced_data['TotalPrice'].sum()),
            'unique_customers': int(diced_data['CustomerID'].nunique()),
            'total_quantity': int(diced_data['Quantity'].sum())
        }

        return jsonify(result)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/olap/rollup', methods=['POST', 'OPTIONS'])
def olap_rollup():
    """OLAP Roll-up operation"""
    if request.method == 'OPTIONS':
        return make_response('', 200)

    try:
        data = request.json
        dimension = data.get('dimension', 'country')

        df = GLOBAL_DATA['df']

        if dimension == 'country':
            rollup = df.groupby('Country').agg({
                'TotalPrice': 'sum',
                'InvoiceNo': 'nunique',
                'CustomerID': 'nunique',
                'Quantity': 'sum'
            }).reset_index()
            rollup.columns = ['Country', 'TotalRevenue', 'TotalOrders', 'UniqueCustomers', 'TotalQuantity']

        elif dimension == 'customer':
            rollup = df.groupby('CustomerID').agg({
                'TotalPrice': 'sum',
                'InvoiceNo': 'nunique',
                'Quantity': 'sum',
                'StockCode': 'nunique'
            }).reset_index()
            rollup.columns = ['CustomerID', 'TotalRevenue', 'TotalOrders', 'TotalQuantity', 'UniqueProducts']

        elif dimension == 'product':
            rollup = df.groupby(['StockCode', 'Description']).agg({
                'TotalPrice': 'sum',
                'Quantity': 'sum',
                'CustomerID': 'nunique',
                'UnitPrice': 'mean'
            }).reset_index()
            rollup.columns = ['StockCode', 'Description', 'TotalRevenue', 'TotalQuantity', 'UniqueCustomers', 'AvgPrice']

        rollup = rollup.sort_values(rollup.columns[2], ascending=False)

        return jsonify({
            'data': rollup.head(50).to_dict('records'),
            'total': len(rollup)
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [16]:
import pandas as pd

try:
    customer_features = pd.read_csv('customer_features.csv')
    print("Sample CustomerIDs:")
    display(customer_features['CustomerID'].sample(5).tolist())
except FileNotFoundError:
    print("customer_features.csv not found. Please run the data processing pipeline first.")

try:
    df_preprocessed = pd.read_csv('online_retail_preprocessed.csv')
    print("\nSample InvoiceNos:")
    display(df_preprocessed['InvoiceNo'].sample(5).tolist())
    print("\nSample StockCodes:")
    display(df_preprocessed['StockCode'].sample(5).tolist())
except FileNotFoundError:
    print("online_retail_preprocessed.csv not found. Please run the data processing pipeline first.")

Sample CustomerIDs:


[14066, 15952, 17402, 15068, 13911]


Sample InvoiceNos:


[550526, 546394, 570210, 551056, 577056]


Sample StockCodes:


['23005', '22851', '21586', '23340', '79066K']

# ===============================================
# PART 9: NGROK SETUP AND API LAUNCH
# ===============================================

In [ ]:
import threading # Import the threading module

def setup_ngrok_and_run():
    """Setup ngrok tunnel and run Flask API"""
    print("\n" + "=" * 80)
    print("SETTING UP NGROK TUNNEL")
    print("=" * 80)

    # Check if running in Colab
    try:
        from google.colab import userdata
        IN_COLAB = True
        print("✅ Running in Google Colab")
    except ImportError:
        IN_COLAB = False
        print("ℹ️ Not running in Colab")

    # Import ngrok
    try:
        from pyngrok import ngrok
    except ImportError:
        print("Installing pyngrok...")
        import subprocess
        subprocess.check_call(['pip', 'install', '-q', 'pyngrok'])
        from pyngrok import ngrok

    # Setup ngrok authentication
    if IN_COLAB:
        try:
            # Try to get token from Colab secrets
            authtoken = userdata.get('NGROK_AUTHTOKEN')
            if authtoken:
                ngrok.set_auth_token(authtoken)
                print("✅ Ngrok authtoken set from Colab secrets")
            else:
                # Prompt for token
                print("\n" + "="*60)
                print("NGROK AUTHENTICATION REQUIRED")
                print("="*60)
                print("\nTo get your ngrok authtoken:")
                print("1. Go to: https://dashboard.ngrok.com/signup")
                print("2. Sign up for a free account")
                print("3. Copy your authtoken from the dashboard")
                print("="*60)

                authtoken = input("\nEnter your ngrok authtoken: ").strip()
                if authtoken:
                    ngrok.set_auth_token(authtoken)
                    print("✅ Ngrok authtoken set successfully")
                else:
                    print("⚠️ No authtoken provided. API will run locally only.")

        except Exception as e:
            print(f"⚠️ Could not set ngrok token: {e}")

    # Load data before starting server
    if not load_all_data():
        print("❌ Failed to load data. Please run the pipeline first.")
        return

    # Kill any existing tunnels
    ngrok.kill()

    # Start Flask in a thread
    from threading import Thread

    def run_flask():
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

    flask_thread = Thread(target=run_flask)
    flask_thread.daemon = True
    flask_thread.start()

    # Give Flask time to start
    time.sleep(3)

    # Create ngrok tunnel
    try:
        public_url = ngrok.connect(5000, bind_tls=True)

        print("\n" + "=" * 80)
        print("🚀 E-COMMERCE RECOMMENDATION API IS LIVE!")
        print("=" * 80)
        print(f"\n📱 Public URL: {public_url}")
        print(f"🌐 Copy this URL to use in the HTML interface")
        print("\n📝 API Endpoints:")
        print(f"  • GET  {public_url}/api/test - Test connection")
        print(f"  • GET  {public_url}/api/health - Health check")
        print(f"  • GET  {public_url}/api/stats - Get statistics")
        print(f"  • GET  {public_url}/api/products - Get products")
        print(f"  • GET  {public_url}/api/customers/<id> - Get customer profile")
        print(f"  • POST {public_url}/api/recommend - Get recommendations")
        print(f"  • POST {public_url}/api/classify - Classify transaction")
        print(f"  • POST {public_url}/api/olap/slice - OLAP slice")
        print(f"  • POST {public_url}/api/olap/dice - OLAP dice")
        print(f"  • POST {public_url}/api/olap/rollup - OLAP rollup")
        print("\n" + "=" * 80)

        # Save URL to file
        with open('api_url.txt', 'w') as f:
            f.write(str(public_url))
        print(f"\n✅ API URL saved to 'api_url.txt'")

        # Keep the script running
        print("\n⚠️ Keep this cell running to maintain the API connection")
        print("Press Ctrl+C to stop the server")

        while True:
            time.sleep(1)

    except KeyboardInterrupt:
        print("\n\n🛑 Shutting down server...")
        ngrok.kill()

    except Exception as e:
        print(f"\n❌ Error creating ngrok tunnel: {e}")
        print("The API is still running locally at http://localhost:5000")

# Start the API
setup_ngrok_and_run()


SETTING UP NGROK TUNNEL
✅ Running in Google Colab
✅ Ngrok authtoken set from Colab secrets

LOADING DATA FOR API
✅ All data and models loaded successfully
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



🚀 E-COMMERCE RECOMMENDATION API IS LIVE!

📱 Public URL: NgrokTunnel: "https://blotchier-unlovely-isa.ngrok-free.dev" -> "http://localhost:5000"
🌐 Copy this URL to use in the HTML interface

📝 API Endpoints:
  • GET  NgrokTunnel: "https://blotchier-unlovely-isa.ngrok-free.dev" -> "http://localhost:5000"/api/test - Test connection
  • GET  NgrokTunnel: "https://blotchier-unlovely-isa.ngrok-free.dev" -> "http://localhost:5000"/api/health - Health check
  • GET  NgrokTunnel: "https://blotchier-unlovely-isa.ngrok-free.dev" -> "http://localhost:5000"/api/stats - Get statistics
  • GET  NgrokTunnel: "https://blotchier-unlovely-isa.ngrok-free.dev" -> "http://localhost:5000"/api/products - Get products
  • GET  NgrokTunnel: "https://blotchier-unlovely-isa.ngrok-free.dev" -> "http://localhost:5000"/api/customers/<id> - Get customer profile
  • POST NgrokTunnel: "https://blotchier-unlovely-isa.ngrok-free.dev" -> "http://localhost:5000"/api/recommend - Get recommendations
  • POST NgrokTunnel: "h

INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 16:59:57] "OPTIONS /api/test HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 16:59:57] "GET /api/test HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 16:59:59] "OPTIONS /api/stats HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 16:59:59] "OPTIONS /api/products?page=1&per_page=12&search= HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 17:00:00] "GET /api/stats HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 17:00:00] "GET /api/products?page=1&per_page=12&search= HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 17:00:08] "OPTIONS /api/test HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 17:00:09] "GET /api/test HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 17:00:11] "OPTIONS /api/products?page=1&per_page=12&search= HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 17:00:11] "OPTIONS /api/stats HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2025 17:00:11] "GET /api